<a href="https://colab.research.google.com/github/dframbaut/web-scrapping-deepear/blob/main/Local%20RD/planificacion_y_desarrollo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
!pip install bs4

In [ ]:
!apt-get update
!apt-get install -y ca-certificates

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,517 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [3,448 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-secur

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

**Extraer id de la pagina principal**


In [ ]:
url='https://adn.gob.do/transparencia/planificacion-y-desarrollo/'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

response=requests.get(url,headers=headers,verify=False)
soup=BeautifulSoup(response.text,'html.parser')

a_tags=soup.find_all('a',class_='wpfdcategory catlink')

# Extraer el atributo "data-idcat" de las etiquetas encontradas
ids_list = [tag.get('data-idcat') for tag in a_tags]

# Imprimir los valores extraídos
print("IDCAT encontrados:", ids_list)



/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


IDCAT encontrados: ['1584', '1859', '109', '111', '343']


**Extraer id de las demas secciones de cada id principal**

In [ ]:
import requests
from bs4 import BeautifulSoup
import re

# URL base
base_url = "https://adn.gob.do/transparencia/wp-admin/admin-ajax.php"

# Parámetros base para la solicitud
base_params = {
    "juwpfisadmin": "false",
    "action": "wpfd",
    "task": "categories.display",
    "view": "categories",
    "top": "108"
}

def extraer_term_taxonomy_id(base_url, params, exclude_ids):
    """
    Extrae los valores de term_taxonomy_id para un ID específico,
    excluyendo IDs ya procesados.

    :param base_url: URL base para realizar la solicitud.
    :param params: Diccionario con los parámetros de la solicitud.
    :param exclude_ids: Conjunto de IDs que deben ser excluidos.
    :return: Lista de nuevos term_taxonomy_id encontrados.
    """
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
    }

    try:
        # Realizar solicitud GET
        response = requests.get(base_url, params=params, headers=headers, verify=False)
        response.raise_for_status()  # Genera excepción si el código no es 200

        # Parsear el HTML de la respuesta
        soup = BeautifulSoup(response.text, 'html.parser')

        # Convertir el contenido en texto para buscar con expresiones regulares
        page_text = soup.get_text()

        # Expresión regular para buscar valores de term_taxonomy_id
        term_taxonomy_ids = re.findall(r'"term_taxonomy_id":(\d+)', page_text)

        # Filtrar IDs excluidos
        new_ids = [id_val for id_val in term_taxonomy_ids if id_val not in exclude_ids]

        return new_ids

    except requests.exceptions.RequestException as e:
        print(f"Error durante la solicitud para ID {params['id']}: {e}")
        return []
    except Exception as e:
        print(f"Error al procesar los datos para ID {params['id']}: {e}")
        return []

# Iteración para obtener IDs adicionales
processed_ids = set(ids_list)  # Conjunto para rastrear IDs ya procesados
idcat_values2 = []  # Lista para almacenar los resultados iterativos
iteration_count = 1  # Contador de iteraciones

while True:
    print(f"\nIteración {iteration_count}")
    nuevos_ids = []

    # Iterar sobre los IDs actuales y buscar nuevos term_taxonomy_id
    for id_val in ids_list:
        params = base_params.copy()
        params["id"] = id_val

        # Extraer term_taxonomy_id excluyendo los procesados
        ids_extraidos = extraer_term_taxonomy_id(base_url, params, processed_ids)
        nuevos_ids.extend(ids_extraidos)

    # Eliminar duplicados y actualizar el conjunto de IDs procesados
    nuevos_ids = list(set(nuevos_ids))  # Eliminar duplicados
    nuevos_ids = [id_val for id_val in nuevos_ids if id_val not in processed_ids]

    if not nuevos_ids:
        print("No se encontraron nuevos IDs. Finalizando.")
        break

    # Actualizar los datos para la siguiente iteración
    idcat_values2.append(nuevos_ids)
    processed_ids.update(nuevos_ids)
    ids_list = nuevos_ids
    iteration_count += 1

# Fusionar todas las listas en una sola, eliminando duplicados
final_idcat_values = list(set([item for sublist in idcat_values2 for item in sublist] + list(processed_ids)))

# Mostrar los resultados finales
print("\nResultados finales:")
print(f"Lista combinada de IDCAT: {final_idcat_values}")




Iteración 1


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRe


Iteración 2


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRe


Iteración 3


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRe


Iteración 4


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRe

No se encontraron nuevos IDs. Finalizando.

Resultados finales:
Lista combinada de IDCAT: ['771', '780', '769', '772', '880', '778', '1574', '776', '775', '1464', '1695', '783', '784', '475', '111', '109', '781', '777', '782', '476', '477', '770', '1853', '481', '633', '774', '343', '1576', '479', '1573', '1859', '779', '1575', '1584', '1674', '773', '932', '480', '663', '879']


In [ ]:
print(len(final_idcat_values))

40


In [ ]:
ids_1 = [1584, 1859, 109, 111, 1573, 1574, 477, 932, 475, 476, 663, 769, 770, 771, 343, 1674, 1695, 1853, 633, 880, 879, 1575, 1464, 1576, 479, 481, 480, 772, 773, 776, 775, 774, 777, 778, 779, 780, 781, 782, 783, 784]  # Agrega los IDs que necesites

In [ ]:
common_elements = list(set(final_idcat_values) & set(ids_1))
print("Elementos comunes:", common_elements)

Elementos comunes: []


Código

In [ ]:
import requests
import json
import unicodedata  # Importar unicodedata para la normalización del texto

# URL base de la API
api_url = "https://adn.gob.do/transparencia/wp-admin/admin-ajax.php"

# Lista de IDs que deseas recorrer
ids = final_idcat_values

# Parámetros base de la solicitud
base_params = {
    "juwpfisadmin": "false",
    "action": "wpfd",
    "task": "files.display",
    "view": "files",
    "page": 1,
    "orderCol": "title",
    "orderDir": "desc"
}

nombres = []
link = []
fecha_pub = []
fecha_vigor = []
epigrafe = []

# Función para normalizar texto (remover tildes y convertir a minúsculas)
def normalize_text(text):
    text = unicodedata.normalize('NFD', text)
    text = ''.join(c for c in text if unicodedata.category(c) != 'Mn')
    return text.lower()

# Función para procesar cada ID
def fetch_files_by_id(api_url, ids, base_params):
    all_results = []  # Lista para almacenar resultados de todas las IDs

    for current_id in ids:
        print(f"Procesando ID: {current_id}...")

        # Actualizar los parámetros con el ID actual
        params = base_params.copy()
        params["id"] = current_id
        params["rootcat"] = current_id

        page = 1
        while True:
            params["page"] = page
            response = requests.get(api_url, params=params, verify=False)

            if response.status_code == 200:
                data = response.json()
                archivos = data.get("files", [])

                if not archivos:
                    print(f"No hay más archivos para ID: {current_id}.")
                    break

                # Agregar los archivos procesados a la lista de resultados
                for archivo in archivos:
                    post_title = archivo.get("post_title", "Sin título")
                    normalized_title = normalize_text(post_title)
                    nombres.append(normalized_title)
                    epigrafe.append(normalized_title)
                    created_time = archivo.get("created_time", "Sin fecha")
                    fecha_pub.append(created_time)
                    modified_time = archivo.get("modified_time", "Sin fecha")
                    fecha_vigor.append(modified_time)
                    linkdownload = archivo.get("linkdownload", "Sin enlace")
                    link.append(linkdownload)

                    # Agregar solo los campos requeridos al resultado
                    all_results.append({
                        "post_title": normalized_title,
                        "created_time": created_time,
                        "modified_time": modified_time,
                        "linkdownload": linkdownload
                    })

                page += 1  # Avanzar a la siguiente página
            else:
                print(f"Error al procesar ID: {current_id}. Código de estado: {response.status_code}")
                break

    return all_results

# Ejecutar la función para procesar los IDs
resultados = fetch_files_by_id(api_url, ids, base_params)

# Mostrar los resultados obtenidos
print("Archivos obtenidos:")
print(json.dumps(resultados, indent=4))


Procesando ID: 771...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No hay más archivos para ID: 771.
Procesando ID: 780...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No hay más archivos para ID: 780.
Procesando ID: 769...


...

No hay más archivos para ID: 480.
Procesando ID: 663...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No hay más archivos para ID: 663.
Procesando ID: 879...


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'adn.gob.do'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


No hay más archivos para ID: 879.
Archivos obtenidos:
[
    {
        "post_title": "plan operativo anual(poa) 2018",
        "created_time": "2023-04-20 15:23:00",
        "modified_time": "2023-04-20 15:24:48",
        "linkdownload": "https://adn.gob.do/transparencia/download/771/2018/4517/plan-operativo-anualpoa-2018.pdf"
    },
    {
        "post_title": "simon bolivar-acta de asamblea",
        "created_time": "2023-04-20 17:16:37",
        "modified_time": "2023-04-20 17:16:37",
        "linkdownload": "https://adn.gob.do/transparencia/download/780/actas-asambleas-circunscripcion-3-ano-2010/4559/simon-bolivar-acta-de-asamblea.pdf"
    },
    {
        "post_title": "listado de participantes villa francisca",
        "created_time": "2023-04-20 17:15:00",
        "modified_time": "2023-04-20 18:34:41",
        "linkdownload": "https://adn.gob.do/transparencia/download/780/actas-asambleas-circunscripcion-3-ano-2010/4556/listado-de-participantes-villa-francisca.pdf"
    },
    {
 

Listas

In [ ]:
conteo_nombres=len(nombres)
print(conteo_nombres)
conteo_fecha_pub=len(fecha_pub)
print(conteo_fecha_pub)
conteo_fecha_vigor=len(fecha_vigor)
print(conteo_fecha_vigor)
conteo_link=len(link)
print(conteo_link)
conteo_epigrafe=len(epigrafe)
print(conteo_epigrafe)

167
167
167
167
167


Dataframe

In [ ]:
from sre_constants import NOT_LITERAL
# Crear un DataFrame con los datos recopilados
data = {
    "title": nombres,
    "external_link": link,
    "created_at": fecha_pub,
    "update_at": fecha_vigor,
    'summary':epigrafe
}

df = pd.DataFrame(data)

# Agregar columnas con los valores especificados
df['Tipo'] = None
df['entity']='Alcaldia del distrito Nacional'
df['classification_id']=None
df['rtype_id']='13'
df['gtype']='link'
df['is_active']=True
columnas_ordenadas = ['title','summary','update_at','external_link','entity','created_at','classification_id','rtype_id','gtype','is_active']

df = df[columnas_ordenadas]



# Mostrar el DataFrame resultante
print(df)

                                                 title  \
0                 memorias institucionales 2022 – 2023   
1                                   memorias 2020-2021   
2                         memoria de gestion 2023-2024   
3    memoria de gestion 2021 - 2022, alcaldia del d...   
4    memoria de gestion 2019 - 2020, alcaldia del d...   
..                                                 ...   
162         ley_no_170-07 de presupuesto participativo   
163            informe obras presupuesto participativo   
164            informe obras presupuesto participativo   
165      dirigentes comunitarios circunscripciones 123   
166      asambleas barriales presupuesto participativo   

                                               summary            update_at  \
0                 memorias institucionales 2022 – 2023  2023-07-27 20:18:50   
1                                   memorias 2020-2021  2021-12-13 16:07:44   
2                         memoria de gestion 2023-2024  2024-11-25

In [ ]:
# Exportar el DataFrame a un archivo CSV
df.to_csv("planificacion_y_desarrollo.csv", index=False, encoding='utf-8-sig')

print("Archivo CSV generado exitosamente.")

Archivo CSV generado exitosamente.


In [ ]:
from google.colab import files

# Descargar el archivo generado
files.download("planificacion_y_desarrollo.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>